In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib notebook

# Introduction to Data Storage

##### Version 0.1

***

By AA Miller (Northwestern/CIERA)  
15 July 2022

[Session 15](https://github.com/LSSTC-DSFP/LSSTC-DSFP-Sessions/tree/main/Sessions/Session15) is primarily concerned with handling our data with efficiency.

Ideally, for any and every task we want to devire solutions that operate *faster*. 

This can be accomplished many different ways:

$~~~~~~$build algorithms that execute faster

$~~~~~~$spread calculations over many different computers simultaneously

$~~~~~~$find a compact storage solution for the data so it can be accessed more quickly

In our introduction to this session we will start with data storage, and discuss fast algorithms as a challenge problem. 

Random notes - Scorcese is in the data, Ace Ventura is not, Godfather is not, 

## Problem 1) IMDb Data

Throughout the session we will use information from the [Internet Movie Database (IMDb)](https://www.imdb.com/) to illustrate various principles regarding data storage.

For this notebook, we will use a [google sheets](https://docs.google.com/spreadsheets/d/1B-C7uJFrVNGpAXsGE6_xymfFVSKhwnIsI_RewkkmGa0/edit?usp=sharing) spreadsheet to examine this data (later in the week we will explore the same data stored in a database). 

A quick note on the provenance of this data. The files we have used to populate this data set are from [this website](https://relational.fit.cvut.cz/dataset/IMDb) and it may not be a list of every single movie on IMDb.

**Problem 1a**

Open the google sheet.$^\dagger$ What information is stored in the `movies` sheet of this file? 

How many movies are listed? 

$^\dagger$Note – the link points to a "view" of the data, you may find it useful to copy the file so that you have write access. 

*Write your answer here*

The `movies` sheet contains 4 columns: id, name, year, and rank. `id` appears to be a running index counting the movies, `name` is the title of the movie (and the file appears to be organized alphabetically by name), `year` is the release date for the film, and `rank` is the user score (10 = really good) for the film on IMDb. 

There are 388269 movies in the file. This can (with an investment of time) be determined by scrolling all the way to the end of the file and seeing the top index. We can be more efficient and use the built-in count function to determine the number of movies `=COUNTA(B2:B10000000)`.

**Problem 1b**

What information is stored in the `directors` sheet? 

How many directors are there? 

*Write your answer here*

The `directors` sheet includes 3 columns: id, First Name, Last Name. `id` is a running index, and the name columns are self-explanatory. 

There are 86880 directors total in this sheet. 

**Problem 1c**

What information is stored in the `movies_directors` sheet?

How many rows are there? Does this make sense? Why?

*Write your answer here*

The `movies_directors` sheet includes 2 columns: `movie_id` and `director_id`. `movie_id` corresponds to the `id` column in the `movies` sheet, while `director_id` correspondss to the `id` in the directors sheet.

There are 371180 total rows in this sheet. This answer does not really make sense – we know there are 388269 movies in total, so I would expect just as many entries here. Perhaps there are a bunch of movies without directors? 

**Problem 1d**

Confirm your column identifications in **1c** by finding your favorite movie and making sure `movies_directors` correctly matches it with the proper director.

*Write your answer here*

My favorite movie is "Wayne's World" (at least it's top 5), and that has movie id = 360290. The director was Penelope Sphereis, who has director id = 75368. In the `movies_directors` sheet, this `movie_id` and `director_id` match up confirming my answer from **1c**. 

**Problem 1e**

What information is stored in the `movies_genres` sheet?

How many rows are there? Does this make sense? Why?

*Write your answer here*

The `movies_genres` sheet includes 2 columns: `movie_id` and `genre`. `movie_id` corresponds to the `id` column in the `movies` sheet, while `genre` is one of several potential genres for the movie.

There are 395119 total rows in this sheet. A quick glance at the sheet reveals two things: (i) there are several movies that do not have an identified genre (e.g., movie_id = 3), and (ii) there are movies that have multiple genres (e.g., movie_id = 8). 

It is hard to know how many rows to expect given these two facts, but given that many films fall into multiple genres it makes sense this sheet has more rows than there are total movies.

## Problem 2) Connections

We started this exercise with a goal of being efficient. And yet, the data have been organized across 4 different files (each sheet is effectively a unique csv file).  

**Problem 2a**

If we wanted to store all the data in a single sheet, how many rows would we need?

*write your answer here*

There are only 88680 directors, so we could make a file that only has 88680 rows and store all the information in the 4 sheets. 

**Problem 2b**

Supposing we did convert everything to a single sheet – how many columns would you need to store all the information in a single sheet? 

*Think about this for at least 30 sec, but you do not have to write down a full answer*

Below you will determine the numbers necessary to calculate the true answers to these problems, but we can get a sense of the order of magnitude in a relatively straightforward fashion. 

We need two columns for first and last name. We need $3N_\mathrm{movie}$ columns where $N_\mathrm{movie}$ is the most movies directed by *any* director. In a single sheet this would be `movie1`, `movie2`, ..., `movieN`, plus the associated `year` and `rank`. From `movies_directors` we see that director 8 has 35 movie credits, so this is *at least* 105 additional columns.

At maximum another $21N_\mathrm{movie}$ columns are required to account for the different genres for each movie. This is likely overly conservative, no film belongs to every genre – let's say 5 is a reasonable number per movie (e.g., Action, Sci-Fi, Fantasy, Comedy, Short). Then this is another 175 columns.

**Problem 2c**

Which has fewer total entries (the combination of a column and a row makes 1 entry) – all the data in one sheet, or the data spread across 4 sheets? 

*write your answer here*

For the four sheets there are: $4 \times 388{,}269 + 3 \times 88{,}680 + 2 \times 371{,}180 + 2 \times 395{,}119 = 3{,}351{,}714$ entries

For a single sheet there are: $282 \times 88{,}680 = 25{,}007{,}760$ entries

What we see here is that it is very inefficient/expensive to record "nothing". 

Suppose there is a director that only has one film credit. In the single sheet solution, we need to record that `movie2` = `NULL`, and `movie3` and so on. This information has a cost (storage), AND, there is essentially nothing learned from said information. 

The reason the data have been organized into several sheets (or shall we call them "data tables"; yeah! let's call them "tables") is that it allows us to store far less information. There are some columns that we otherwise would not need (in a single table there is no need for an `id`, whereas the multi-table solution has at least one id in every table). 

The `id` however is very powerful, as it is what allows the connection between the different tables.

**Problem 2d**

What is the release year for every movie directed by Martin Scorcese? 

*Hint* – do not spend more than 10 min working on this problem (you probably should not even spend a full 10 min). 

*write your answer here*

Scorsese has director `id` = 71645. 

If you are scrolling or use built in functions, you can find that the 40 films that Scorsese has directed are: [7842, 8183, 10702, 13395, 13804, 25192, 27108, 37304, 42328, 45312, 47130, 54209, 56304, 67388, 82967, 123849, 131780, 163603, 163730, 177369, 181766, 183593, 185704, 185751, 193231, 199510, 209158, 212717, 214872, 230947, 230963, 253470, 270971, 316709, 326155, 352863, 362473, 364368, 379931, 382309]

If we copy that list of movie numbers to the `movies` sheet, we can then use a `FILTER` to find the years:  
`=FILTER(C2:C1000000, A2:A1000000 = G10)`  where G10 is 7842. This filter can then be copied to all rows below the first to result in the following years (this is sorted by title alphabetically):

[1985, 1993, 1974, 1992, 1978, 2004, 1987, 1967, 2005, 1972, 1999, 1991, 1995, 1986, 2006, 2002, 1990, 1964, 1974, 1983, 1997, 2004, 1988, 1978, 1988, 1990, 1973, 1995, 1999, 1989, 1977, 1995, 1980, 1970, 1976, 1959, 1963, 1967, 1985, 2003]